# Alexnet 코드 구현(pytorch, tensorflow)

### AlexNet model(pytorch)

In [1]:
import torch
import torch.nn as nn
from typing import Any

try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url
    

__all__=['AlexNet','alexnet']

model_urls={
    'alexnet':'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth'
}

class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 1000) -> None:
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            #Conv1
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            
            #Max Pool1
            nn.MaxPool2d(kernel_size=3,stride=2),
            
            #Conv2
            nn.Conv2d(64, 192, kernel_size=3, stride=2),
            nn.ReLU(inplace=True),
            
            #Max Pool2
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            #Conv3
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            
            #Conv4
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            
            #Conv5
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            
            #Max Pool3
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6,6))
        self.classifier = nn.Sequential(
            #드롭아웃
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )
        
        def forward(self, x: torch.Tensor) -> torch.Tensor:
            x = self.features(x)
            x = self.avgpool(x)
            x = torch.flatten(x, 1)
            x = self.classifier(x)
            return x    
        
def alexnet(pretrained: bool = False, progress:bool = True, **kwargs: Any) -> AlexNet:
    model = AlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],progress=progress)
        model.load_state_dict(state_dict)
    return model
    

### AlexNet model(tensorflow)

In [2]:
import tensorflow as tf

def AlexNet(
    input_shape=None,
    weights=None,
    classes=1000,
    classifier_activation='softmax'
):
    model = tf.keras.Sequential([
        #Conv1
        tf.keras.layers.Conv2D(filters=96,
                              kernel_size=(11,11),
                              strides=4,
                              padding='valid',
                              activation=tf.keras.activations.relu,
                              input_shape=input_shape),
        
        #Max Pool1
        tf.keras.layers.MaxPool2D(pool_size=(3, 3),
                                 strides=2,
                                 padding='valid'),
        tf.keras.layers.BatchNormalization(),
        
        #Conv2
        tf.keras.layers.Conv2D(filters=256,
                              kernel_size=(5,5),
                              strides=1,
                              padding='same',
                              activation=tf.keras.activations.relu),
        
        #Max Pool2
        tf.keras.layers.MaxPool2D(pool_size=(3, 3),
                                 strides=2,
                                 padding='same'),
        tf.keras.layers.BatchNormalization(),
        
        #Conv3
        tf.keras.layers.Conv2D(filters=384,
                              kernel_size=(3,3),
                              strides=1,
                              padding='same',
                              activation=tf.keras.activations.relu),
        
        #Conv4
        tf.keras.layers.Conv2D(filters=384,
                              kernel_size=(3,3),
                              strides=1,
                              padding='same',
                              activation=tf.keras.activations.relu),
        
        #Conv5
        tf.keras.layers.Conv2D(filters=256,
                              kernel_size=(3,3),
                              strides=1,
                              padding='same',
                              activation=tf.keras.activations.relu),
        
        #Max Pool3
        tf.keras.layers.MaxPool2D(pool_size=(3, 3),
                                 strides=2,
                                 padding='same'),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.Flatten(),
        
        #분류층
        tf.keras.layers.Dense(units=4096,
                             activation=tf.keras.activations.relu),
        tf.keras.layers.Dropout(rate=0.2),
        
        tf.keras.layers.Dense(units=4096,
                             activation=tf.keras.activations.relu),
        tf.keras.layers.Dropout(rate=0.2),
        
        tf.keras.layers.Dense(units=classes,
                             activation=tf.keras.activations.softmax)    
    ])
    
    return model